In [46]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

dummy_vocab = ['battery life', 'picture quality', 'size']
vectorizer = CountVectorizer(ngram_range=(1,2), token_pattern=r'\b\w+\b', vocabulary=dummy_vocab)
dummy_corpus = ['The battery life sucks.',
                'The picture quality is great, but poor battery life.',
                'Loved this product, but the size was too large.']
X = vectorizer.fit_transform(dummy_corpus)
df = X.power(0).sum(0).A1 #Calculates df of each feature
df_dict = dict(zip(dummy_vocab, df))
df_dict

{'battery life': 2, 'picture quality': 1, 'size': 1}